In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
from torch.utils.data import TensorDataset, DataLoader

In [3]:
import lightning as pl
from lightning.pytorch.loggers import TensorBoardLogger

import torchmetrics

In [4]:
# config
max_epochs = 4
learning_rate = 0.005

in_features=901
h1 = 800
h2 = 600
h3 = 200
out_features = 128
batch_size = 512

logging_name = 'linear nn2'

project = "chess nn2 linear model 4, my moves, 1 label"
architecture = "Linear network with 2 hidden layers, my moves, 1 label",
dataset = "max rapid games",

In [5]:
# data_training = torch.load('tensor_data/nn1_rapid_games_data_training_09_02_24.pt')
data_training = torch.from_numpy(np.load('np_arrays/nn2_linear_train.npy'))
# print(data_training.shape)
# data_training.to(device)

In [6]:
# data_testing = torch.load('tensor_data/nn1_rapid_games_data_testing_09_02_24.pt')
data_valid = torch.from_numpy(np.load('np_arrays/nn2_linear_valid.npy'))

# print(data_testing.shape)
# data_testing.to(device)

In [7]:
lables_training = torch.from_numpy(np.load('np_arrays/nn2_linear_train_lables.npy'))
# print(lables_training.shape)
# lables_training.to(device)

In [8]:
lables_valid = torch.from_numpy(np.load('np_arrays/nn2_linear_valid_lables.npy'))
# lables_testing = torch.load('tensor_data/nn1_rapid_games_lables_testing_09_02_24.pt')
# print(lables_testing.shape)
# lables_testing.to(device)

In [9]:
# data_training = data_training.half()
# data_testing = data_testing.half()
# lables_training = lables_training.half()
# lables_testing = lables_testing.half()
# lables_training = lables_training.float()
# lables_testing = lables_testing.float()

In [10]:
data_training = data_training.float()
data_valid = data_valid.float()
lables_training = lables_training.long().view(-1)
lables_valid = lables_valid.long().view(-1)

In [11]:
print(data_training.shape)
print(lables_training.shape)
print(data_valid.shape)
print(lables_valid.shape)

torch.Size([134152, 901])
torch.Size([134152])
torch.Size([33538, 901])
torch.Size([33538])


In [12]:
print(data_training.shape)
print(lables_training.shape)
print(data_valid.shape)
print(lables_valid.shape)

torch.Size([134152, 901])
torch.Size([134152])
torch.Size([33538, 901])
torch.Size([33538])


In [13]:
# print(data_valid.dtype)
# print(lables_training.dtype)
# print(lables_valid.dtype)

In [14]:
torch.manual_seed(42)

In [15]:
training_dataset = TensorDataset(data_training, lables_training)
valid_dataset = TensorDataset(data_valid, lables_valid)

In [16]:
data_loader_training = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=7)
data_loader_valid = DataLoader(valid_dataset, batch_size=batch_size, num_workers=7)


In [17]:
class Model(pl.LightningModule):

    def __init__(self, in_features=in_features, h1=h1, h2=h2, h3=h3, out_features=out_features): #out = 64 squares to move to 
        super().__init__()

        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        # self.fc2 = nn.Linear(h1, h2)
        # self.fc3 = nn.Linear(h2, h3)
        # self.fc3 = nn.Linear(h2, h3)
        self.out = nn.Linear(h2, out_features)

        self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=out_features)

    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        x = self.out(x)
        
        return x


    def training_step(self, batch, batch_index):

        x, y = batch
        preds = self.forward(x)
        print(preds)


        loss = F.cross_entropy(preds, y)
        
        accuracy = self.accuracy(preds, y)

        self.log('loss', loss)
        self.log('accuracy', accuracy)

        # self.log('train_acc_step', self.accuracy)

        return loss

    def validation_step(self, batch, batch_index):

        x, y = batch
        preds = self.forward(x)
        loss = F.cross_entropy(preds, y)
        accuracy = self.accuracy(preds, y)
        
        # self.log('train_acc_step', self.accuracy)
        self.log('loss validation', loss)
        self.log('accuracy validation', accuracy)

        return loss
        

    def configure_optimizers(self):

        return torch.optim.Adam(self.parameters(), lr=learning_rate)



In [18]:
model = Model()

In [19]:
logger = TensorBoardLogger("lightning_logs", name=logging_name) #tb logs is a directory?
# trainer = pl.Trainer(max_epochs=20, logger=logger, precision='32')
trainer = pl.Trainer(max_epochs=max_epochs, logger=logger, precision='16-mixed')

# trainer.fit(model, data_loader_training)
trainer.fit(model, data_loader_training, data_loader_valid)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type               | Params
------------------------------------------------
0 | fc1      | Linear             | 721 K 
1 | fc2      | Linear             | 480 K 
2 | out      | Linear             | 76.9 K
3 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.117     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/263 [00:00<?, ?it/s]                           tensor([[-0.0587, -0.0521, -0.0350,  ...,  0.0281, -0.0801,  0.0156],
        [-0.0377, -0.0488, -0.0201,  ...,  0.0035, -0.0715,  0.0136],
        [-0.0739, -0.0432, -0.0215,  ...,  0.0209, -0.0811,  0.0211],
        ...,
        [-0.0612, -0.0364, -0.0286,  ..., -0.0008, -0.0801,  0.0147],
        [-0.0705, -0.0468, -0.0226,  ...,  0.0365, -0.0702,  0.0259],
        [-0.0646, -0.0334, -0.0278,  ...,  0.0088, -0.0758,  0.0139]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddmmBackward0>)
Epoch 0:   0%|          | 1/263 [00:01<04:45,  0.92it/s, v_num=0]tensor([[-0.2922, -0.1670, -0.1516,  ..., -0.2040, -0.3130, -0.1971],
        [-0.2612, -0.1461, -0.1354,  ..., -0.1703, -0.2808, -0.1826],
        [-0.5908, -0.2734, -0.2408,  ..., -0.3855, -0.5522, -0.3965],
        ...,
        [-0.3328, -0.1710, -0.1649,  ..., -0.2430, -0.3481, -0.2389],
        [-0.6865, -0.3455, -0.3330,  ..., -0.4563, -0.6455, -0

`Trainer.fit` stopped: `max_epochs=4` reached.


Epoch 3: 100%|██████████| 263/263 [00:07<00:00, 35.56it/s, v_num=0]


In [20]:
torch.save(model.state_dict(), f'models/nn2_model_h{h1}_h{h2}_epochs_{max_epochs}.pt')